# DataFrame

## Create DataFrame

In [110]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark Course").master("local[*]").getOrCreate()
sc = spark.sparkContext

In [111]:
# list
L = [[0, "Oliver", 30, "London", "January"],\
     [1, "Emily", 26, "Manchester", "February"],\
     [2, "James", 29, "Birmingham", "March"],\
     [3, "Sophia", 35, "Leeds", "April"],\
     [4, "Liam", 22, "Liverpool", "May"],\
     [5, "Charlotte", 28, "Bristol", "June"],\
     [6, "Ethan", 27, "Sheffield", "July"]]

rdd = sc.parallelize(L)

In [112]:
rdd.getNumPartitions()

2

### automatic schema

In [113]:
df = rdd.toDF(["", "Name", "Age", "City", "Month"])
df.printSchema() 
df.show()

root
 |-- : long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- City: string (nullable = true)
 |-- Month: string (nullable = true)

+---+---------+---+----------+--------+
|   |     Name|Age|      City|   Month|
+---+---------+---+----------+--------+
|  0|   Oliver| 30|    London| January|
|  1|    Emily| 26|Manchester|February|
|  2|    James| 29|Birmingham|   March|
|  3|   Sophia| 35|     Leeds|   April|
|  4|     Liam| 22| Liverpool|     May|
|  5|Charlotte| 28|   Bristol|    June|
|  6|    Ethan| 27| Sheffield|    July|
+---+---------+---+----------+--------+



### specify schema

In [114]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([StructField("", IntegerType(), False),\
                     StructField("Name", StringType()),\
                     StructField("Age", IntegerType()),\
                     StructField("City", StringType()),\
                     StructField("Month", StringType())])

In [115]:
df = rdd.toDF(schema)
df.printSchema() 
df.show()

root
 |-- : integer (nullable = false)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Month: string (nullable = true)

+---+---------+---+----------+--------+
|   |     Name|Age|      City|   Month|
+---+---------+---+----------+--------+
|  0|   Oliver| 30|    London| January|
|  1|    Emily| 26|Manchester|February|
|  2|    James| 29|Birmingham|   March|
|  3|   Sophia| 35|     Leeds|   April|
|  4|     Liam| 22| Liverpool|     May|
|  5|Charlotte| 28|   Bristol|    June|
|  6|    Ethan| 27| Sheffield|    July|
+---+---------+---+----------+--------+



## Read data into DataFrame

### read file csv

In [116]:
df = spark.read.option("header", True).option("inferSchema", True).csv("/home/ubuntu/pyspark/data/Tutorial_8/CarData.csv")

In [117]:
df.printSchema() 
df.show()

root
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)

+------+--------+----+--------+-----+
| Brand|   Model|Year|    Type|Price|
+------+--------+----+--------+-----+
| Tesla| Model S|2023|Electric|85000|
|  Ford| Mustang|2021|  Petrol|55000|
|Toyota| Corolla|2020|  Hybrid|30000|
|   BMW|Series 3|2022|  Diesel|45000|
|  Audi|      A4|2019|  Petrol|40000|
+------+--------+----+--------+-----+



In [118]:
from pyspark.sql.types import FloatType

schema_car = StructType([
    StructField("ID", IntegerType(), False),
    StructField("Brand", StringType(), True),
    StructField("Model", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Type", StringType(), True),
    StructField("Price", FloatType(), True)
])

In [119]:
df = spark.read.schema(schema_car).option("delimiter", ",").option("header", True).csv("/home/ubuntu/pyspark/data/Tutorial_8/CarData.csv")

In [120]:
df.printSchema() 
df.show()

root
 |-- ID: integer (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)

+----+--------+-----+----+-----+-----+
|  ID|   Brand|Model|Year| Type|Price|
+----+--------+-----+----+-----+-----+
|NULL| Model S| 2023|NULL|85000| NULL|
|NULL| Mustang| 2021|NULL|55000| NULL|
|NULL| Corolla| 2020|NULL|30000| NULL|
|NULL|Series 3| 2022|NULL|45000| NULL|
|NULL|      A4| 2019|NULL|40000| NULL|
+----+--------+-----+----+-----+-----+



### read file json

In [121]:
df = spark.read.option("multiLine",True).json("/home/ubuntu/pyspark/data/Tutorial_8/CarData.json")
df.printSchema()
df.show()

root
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Price: long (nullable = true)
 |-- Type: string (nullable = true)
 |-- Year: long (nullable = true)

+------+--------+-----+--------+----+
| Brand|   Model|Price|    Type|Year|
+------+--------+-----+--------+----+
| Tesla| Model S|85000|Electric|2023|
|  Ford| Mustang|55000|  Petrol|2021|
|Toyota| Corolla|30000|  Hybrid|2020|
|   BMW|Series 3|45000|  Diesel|2022|
|  Audi|      A4|40000|  Petrol|2019|
+------+--------+-----+--------+----+



In [122]:
df = spark.read.schema(schema_car).option("multiLine",True).json("/home/ubuntu/pyspark/data/Tutorial_8/CarData.json")
df.show()
df.printSchema()

+----+------+--------+----+--------+-------+
|  ID| Brand|   Model|Year|    Type|  Price|
+----+------+--------+----+--------+-------+
|NULL| Tesla| Model S|2023|Electric|85000.0|
|NULL|  Ford| Mustang|2021|  Petrol|55000.0|
|NULL|Toyota| Corolla|2020|  Hybrid|30000.0|
|NULL|   BMW|Series 3|2022|  Diesel|45000.0|
|NULL|  Audi|      A4|2019|  Petrol|40000.0|
+----+------+--------+----+--------+-------+

root
 |-- ID: integer (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)



In [123]:
df.write.parquet("/home/ubuntu/pyspark/data/Tutorial_8/CarData.parquet")

# Manipulating a DataFrame

## Initial Methods and Exploration of a DataFrame

In [124]:
df = spark.read.option("header", True).option("inferSchema", True).option("escape", "\"").csv("/home/ubuntu/pyspark/data/Tutorial_8/AppData.csv")
df.printSchema()
df.show(3)

root
 |-- Application Name: string (nullable = true)
 |-- App Category: string (nullable = true)
 |-- Average Rating: double (nullable = true)
 |-- Number of Reviews: integer (nullable = true)
 |-- App Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- App Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Classification: string (nullable = true)
 |-- App Genres: string (nullable = true)
 |-- Update Date: string (nullable = true)
 |-- Current Version: string (nullable = true)
 |-- Minimum Android Version: string (nullable = true)

+-------------------+--------------+--------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+
|   Application Name|  App Category|Average Rating|Number of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|      Update Date|Current Version|Minimum Android Version

In [125]:
print(df.head(2))
print(df.tail(2))
print(df.first())

[Row(Application Name='Local cup since', App Category='ART_AND_DESIGN', Average Rating=3.8, Number of Reviews=216450, App Size='28.4M', Installs='10+', App Type='Free', Price='0', Content Classification='Teen', App Genres='Art & Design;Creativity', Update Date='November 05, 2022', Current Version='6.3.2', Minimum Android Version='4.2 and up'), Row(Application Name='Player home ago for', App Category='ART_AND_DESIGN', Average Rating=4.5, Number of Reviews=152455, App Size='2.3M', Installs='100,000+', App Type='Free', Price='0', Content Classification='Teen', App Genres='Art & Design', Update Date='December 10, 2024', Current Version='8.8.7', Minimum Android Version='4.2 and up')]
[Row(Application Name='Anyone act against head', App Category='ART_AND_DESIGN', Average Rating=3.7, Number of Reviews=261753, App Size='46.2M', Installs='1,000,000+', App Type='Free', Price='0', Content Classification='Everyone', App Genres='Art & Design', Update Date='August 09, 2023', Current Version='2.0.5',

In [126]:
df_subset = df.sample(True, 0.001)
df_subset.show()

+----------------+--------------+--------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+----------------+---------------+-----------------------+
|Application Name|  App Category|Average Rating|Number of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|     Update Date|Current Version|Minimum Android Version|
+----------------+--------------+--------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+----------------+---------------+-----------------------+
|   Later foreign|ART_AND_DESIGN|           3.9|            78776|   33.8M|     50+|    Paid|  $49|              Everyone|Art & Design;Pret...|October 04, 2022|          6.4.3|             5.0 and up|
+----------------+--------------+--------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+----------------+---------------+--------------------

In [127]:
df.summary("min","13%","mean","max").show()

+-------+--------------------+--------------+-----------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+------------------+---------------+-----------------------+
|summary|    Application Name|  App Category|   Average Rating|Number of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|       Update Date|Current Version|Minimum Android Version|
+-------+--------------------+--------------+-----------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+------------------+---------------+-----------------------+
|    min|Ability account h...|ART_AND_DESIGN|              3.0|              670|   10.0M|  1,000+|    Free|   $1|              Everyone|        Art & Design|    April 01, 2022|          1.0.0|           4.0.3 and up|
|    13%|                NULL|          NULL|              3.3|            33631|    NULL|    NULL|    NULL|  0.0|              

In [128]:
df.describe(["Number of Reviews"]).show()

+-------+-----------------+
|summary|Number of Reviews|
+-------+-----------------+
|  count|             1000|
|   mean|        146585.09|
| stddev|87489.29711714429|
|    min|              670|
|    max|           299969|
+-------+-----------------+



## Explanation of the Fundamental Concept of Column and Basic Manipulations

In [129]:
df.select("Application Name").show(2)
df.select(df["Application Name"]).show(2)

+-------------------+
|   Application Name|
+-------------------+
|    Local cup since|
|Player home ago for|
+-------------------+
only showing top 2 rows

+-------------------+
|   Application Name|
+-------------------+
|    Local cup since|
|Player home ago for|
+-------------------+
only showing top 2 rows



In [130]:
from pyspark.sql.functions import *

df.select("Application Name").show(2)
df.select(df["Application Name"]).show(2)
df.select(col("Application Name")).show(2)

+-------------------+
|   Application Name|
+-------------------+
|    Local cup since|
|Player home ago for|
+-------------------+
only showing top 2 rows

+-------------------+
|   Application Name|
+-------------------+
|    Local cup since|
|Player home ago for|
+-------------------+
only showing top 2 rows

+-------------------+
|   Application Name|
+-------------------+
|    Local cup since|
|Player home ago for|
+-------------------+
only showing top 2 rows



In [131]:
df.select(4 * col("Average Rating")).show(2)

+--------------------+
|(Average Rating * 4)|
+--------------------+
|                15.2|
|                18.0|
+--------------------+
only showing top 2 rows



In [132]:
df_20 = df.withColumn("Rating/20", 4 * col("Average Rating"))
df_20.show(2)

+-------------------+--------------+--------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+
|   Application Name|  App Category|Average Rating|Number of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|      Update Date|Current Version|Minimum Android Version|Rating/20|
+-------------------+--------------+--------------+-----------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+
|    Local cup since|ART_AND_DESIGN|           3.8|           216450|   28.4M|     10+|    Free|    0|                  Teen|Art & Design;Crea...|November 05, 2022|          6.3.2|             4.2 and up|     15.2|
|Player home ago for|ART_AND_DESIGN|           4.5|           152455|    2.3M|100,000+|    Free|    0|                  Teen|        Art & D

In [133]:
df_renamed = df_20.withColumnRenamed("Number of Reviews", "Nbr of Reviews")
df_renamed.show(2)

+-------------------+--------------+--------------+--------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+
|   Application Name|  App Category|Average Rating|Nbr of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|      Update Date|Current Version|Minimum Android Version|Rating/20|
+-------------------+--------------+--------------+--------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+
|    Local cup since|ART_AND_DESIGN|           3.8|        216450|   28.4M|     10+|    Free|    0|                  Teen|Art & Design;Crea...|November 05, 2022|          6.3.2|             4.2 and up|     15.2|
|Player home ago for|ART_AND_DESIGN|           4.5|        152455|    2.3M|100,000+|    Free|    0|                  Teen|        Art & Design|December 

In [134]:
df_renamed.where((col("Rating/20") > 15) | (col("Rating/20") < 5)).show(2)

+-------------------+--------------+--------------+--------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+
|   Application Name|  App Category|Average Rating|Nbr of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|      Update Date|Current Version|Minimum Android Version|Rating/20|
+-------------------+--------------+--------------+--------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+
|    Local cup since|ART_AND_DESIGN|           3.8|        216450|   28.4M|     10+|    Free|    0|                  Teen|Art & Design;Crea...|November 05, 2022|          6.3.2|             4.2 and up|     15.2|
|Player home ago for|ART_AND_DESIGN|           4.5|        152455|    2.3M|100,000+|    Free|    0|                  Teen|        Art & Design|December 

In [135]:
df_used = df_renamed.withColumn("used", when(col("Nbr of Reviews") >= 10000, True).\
                                        otherwise(False))
df_used.show(3)

+-------------------+--------------+--------------+--------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+----+
|   Application Name|  App Category|Average Rating|Nbr of Reviews|App Size|Installs|App Type|Price|Content Classification|          App Genres|      Update Date|Current Version|Minimum Android Version|Rating/20|used|
+-------------------+--------------+--------------+--------------+--------+--------+--------+-----+----------------------+--------------------+-----------------+---------------+-----------------------+---------+----+
|    Local cup since|ART_AND_DESIGN|           3.8|        216450|   28.4M|     10+|    Free|    0|                  Teen|Art & Design;Crea...|November 05, 2022|          6.3.2|             4.2 and up|     15.2|true|
|Player home ago for|ART_AND_DESIGN|           4.5|        152455|    2.3M|100,000+|    Free|    0|                  Teen|        Ar